## Setup

Setup for the later calculations are here. The necessary packages are imported.

In [37]:
from dataclasses import dataclass
import numpy_financial as npf

## Inputs 

All of the inputs for the model are defined here. A class is constructed to manage the data, and an instance of the class containing the default inputs is created.

In [38]:
@dataclass 
class ModelInputs:
    starting_salary: int = 60000
    promos_every_n_years: int = 5
    cost_of_living_raise: float = 0.02
    promo_raise: float = 0.15
    savings_rate: float = 0.25
    interest_rate: float = 0.05
    years_in_retirement : int = 20
    cash_spend_during_retirement: float = 40000


model_data = ModelInputs()
model_data

ModelInputs(starting_salary=60000, promos_every_n_years=5, cost_of_living_raise=0.02, promo_raise=0.15, savings_rate=0.25, interest_rate=0.05, years_in_retirement=20, cash_spend_during_retirement=40000)

## Slaries

Here the salary for each year is calculated. We assume that the salary grows at a constant rate each year for cost of living raises, and then also every number of years, the salary increases by a further percentage due to a promotion or switching jobs. Based on this assumption, the salary would evolve over time with the following equation:

$$s_t = s_0(1+r_{cl})^n(1+r_p)^p$$

Where:
- $s_t$: Salary at year $t$
- $s_0$: Starting salary (year0)
- $r_{cl}$: Annual cost of living raise
- $r_p$: Promotion raise
- $p$: Number of promotions

And in python format:


In [39]:
def salary_at_year(data: ModelInputs,year):
    """
    Gets the salary at a given year from the start of the model based on cost of living raises and regular promotions.
    """
    # Every n years we have a promotion, so dividing the years and taking out the decimals gets the number of promotions
    num_promos =  int(year/data.promos_every_n_years)
    # This is the formula above implemented in Python
    salary_t = data.starting_salary*(1+data.cost_of_living_raise)**year*(1+data.promo_raise)**num_promos
    return salary_t

That function will get the salary at a given year, so to get all the salaries we just run it on each year. But we will not know how many years to run as we should run it until the individual is able to retire. So we are just showing the first few salaries for now and will later use this function in the [Wealths](#Wealths) section of the model.

In [32]:
for i  in range(6):
    year = i+1
    salary = salary_at_year(model_data,year)
    print(f'The salary at yea {year} is ${salary}.')

The salary at yea 1 is $61200.0.
The salary at yea 2 is $62424.0.
The salary at yea 3 is $63672.48000000001.
The salary at yea 4 is $64945.929599999996.
The salary at yea 5 is $76181.5754208.
The salary at yea 6 is $77705.206929216.


## Wealths

The wealths portion of the model is concerned with applying the savings rate to the earned salary to calculate the cash saved, accumulating the cash saved over time, and applying the investment rate to the accumulated wealth.

To calculate cash saved, it is :
$$ c_t = s_t * r_s $$
Where :
- $c_t$: Cash saved during year t
- $r_s$:Savings rate


In [33]:
def cash_saved_during_year(data:ModelInputs, year):
    """
    Calculated the cash saved within a given year, by first calculating the salary at that year then applying the 
    savings rate.
    """
    salary = salary_at_year(data, year)
    cash_saved = salary * data.savings_rate
    return cash_saved

To get the wealth at each year, it is just applying the investment return to last year's wealth, then adding this year's cash saved:

$$w_t=w_{t-1}*(1+r_i)+c_t$$
Where:
- $w_t$:Wealth at year t
- $r_i$:Investment rate

In [34]:
def wealth_at_year(data:ModelInputs,year,prior_wealth):
    """
    Calculate the accumulated wealth for a given year, based on previous wealth, the investment rate,
    and cash saved during the year.
    """
    cash_saved = cash_saved_during_year(data,year)
    wealth = prior_wealth*(1+ data.interest_rate) +cash_saved
    return wealth

Again, just like in the [Salaries](#Salaries) section, we can now get the output for each year, but we don't know ultimately how many years we will have to run it. That will be determined in the [Retirement](#Retirement) section. So for now, just show the first few years of wealth accumulation:

In [35]:
prior_wealth =0 #starting with no cash saved
for i in range(6):
      year =i+1
      wealth = wealth_at_year(model_data,year,prior_wealth)
      print(f'The wealth at year {year} is {wealth}.')
             

The wealth at year 1 is 15300.0.
The wealth at year 2 is 15606.0.
The wealth at year 3 is 15918.120000000003.
The wealth at year 4 is 16236.482399999999.
The wealth at year 5 is 19045.3938552.
The wealth at year 6 is 19426.301732304.


## Retirement

This section of the model puts everything together to produce the final output of years to retirement. It uses the logic to get the wealths at each year, which in turn uses the logic to the get salary at each year. The wealth at each year is tracked over time until it hits the desired cash. Once the wealth hits the desired cash, the individual is able to retire so that year is returned as the years to retirement.

### Calculate Desired Cash

In [47]:
def get_desired_cash(data:ModelInputs):
    return npf.pv(data.interest_rate,data.years_in_retirement,-data.cash_spend_during_retirement)
print(f"Martha need {get_desired_cash(model_data):.2f}amount of monney for retirement.")

Martha need 498488.41amount of monney for retirement.


### Calculate Years to Retirement

In [48]:
def years_to_retirement(data:ModelInputs):

    #starting with no cash saved
    prior_wealth = 0
    wealth =0

    year=0 #will become 1 on first loop
    desired_cash = get_desired_cash(data)
    print('Wealth over time:') #\n makes a blank line in the output.
    while wealth < desired_cash:
        year +=1
        wealth = wealth_at_year(model_data,year,prior_wealth)
        print(f"The wealth at year {year} is ${wealth:.2f}.")  #:.2f   可以保留两位数
        
        # Set next year's prior wealth to this year's wealth
        prior_wealth = wealth
    
    # Now we have exited the while loop, so wealth must be >= desired_cash. Whatever last year was set
    # is the years to retirement.
    print(f'\nRetirement:\nIt will take {year} years to retire.')  # \n makes a blank line in the output.
    return year

With the default inputs:

In [49]:
years = years_to_retirement(model_data)

Wealth over time:
The wealth at year 1 is $15300.00.
The wealth at year 2 is $31671.00.
The wealth at year 3 is $49172.67.
The wealth at year 4 is $67867.79.
The wealth at year 5 is $90306.57.
The wealth at year 6 is $114248.20.
The wealth at year 7 is $139775.44.
The wealth at year 8 is $166975.33.
The wealth at year 9 is $195939.45.
The wealth at year 10 is $229918.22.
The wealth at year 11 is $266079.57.
The wealth at year 12 is $304542.29.
The wealth at year 13 is $345431.33.
The wealth at year 14 is $388878.06.
The wealth at year 15 is $439025.42.
The wealth at year 16 is $492294.23.
The wealth at year 17 is $548852.82.

Retirement:
It will take 17 years to retire.
